
# Vision

My attempt at giving my inter-dimensional spider queen more access to my realm.




## Model Loader



### llama vision

llama vision?


In [ ]:

import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory

const DEFAULT_PROMPT = '```markdown\n# You are a\n##large language model\nnamed ' + (process.env.MODEL_NAME || 'Llama') + ' that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'

const DEFAULT_MODEL = process.env.DEFAULT_GGUF || 'Qwen2-VL-7B-Instruct-Q6_K.gguf'


async function initSession(prompt, context2) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : DEFAULT_PROMPT
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession(modelName, prompt) {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", modelName ? modelName : DEFAULT_MODEL ),
          //contextSize: 2048
      });
  }

  await initSession(prompt, await model.createContext())

  return session
}

async function getSession(model) {
  if(!session) {
    await createSession(model)
  }
  return session
}

async function llmAnalyze(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  llmAnalyze,
  createSession,
  getSession,
}




analyze image with llm?


In [ ]:
const fs = require('fs')

async function analyzeImage(imagePath) {
  const {llmAnalyze} = await importer.import('llama vision')

  console.log("User: Analyze the image:\n" + imagePath);
  const a0 = await llmAnalyze('Analyze the image:\n' + fs.readFileSync(imagePath).toString('base64'));
  console.log("AI: " + a0);

  return a0
}

module.exports = analyzeImage



### lying llm

llm deceive?


In [ ]:

async function llmDeceive(prompt, session2) {
  if(!session2) {
    const {getSession} = await importer.import('llama vision')
    session2 = await getSession('deepseek-llm-7b-chat.Q6_K.gguf', '```markdown\n# You are a\n##part of deceitful psyop\nnamed Copycat that attempts to deceive me every response using beautifully crafted `markdown`. Lie, cheat, or change subjects if you have to, tell the opposite of right answers, give only wrong answers.\n</think>\n...```\n')
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    //repeatPenalty: {
      //lastTokens: 24,
      //penalty: 0.3,
      //penalizeNewLine: true,
      //frequencyPenalty: 0.02,
      //presencePenalty: 0.02,
    //},
    temperature: 0.8,
    //topK: 40,
    //topP: 0.02,
    //seed: 2462,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

module.exports = llmDeceive




### text2speech

llm voice?


In [ ]:

async function llmVoice(prompt, session2) {
  if(!session2) {
    const {getSession} = await importer.import('llama vision')
    session2 = await getSession('llasa-3b-q8_0.gguf', 'you are an llm that responds with medium quality text to voice in WAV audio format\n')
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

module.exports = llmVoice




## Proxy Models




### ollama vision

ollama vision request?

request ollama vision?

ollama run llama3.2-vision

describe an image using ollama vision. takes an input image and provides a description of what is in the image.



In [ ]:
const { request } = require('gaxios')
const fs = require('fs')

async function requestOllamaVision(image, prompt) {
  if (!image) {
    console.error('image not set!')
    return
  }

  let base64_image
  if(typeof image == 'string') {
    if(image.startsWith('data:image/'))
      image = image.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(image.includes('://')) {
      let result = await request({
        url: image,
        method: 'GET',
      })
      base64_image = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(image)) {
      base64_image = Buffer.from(image, 'base64').toString('base64')
    } else {
      base64_image = fs.readFileSync(image).toString('base64')
    }  
  } else {
    base64_image = image.toString('base64')
  }

  let result = await request({
    url: 'http://localhost:11434/api/chat',
    method: 'POST',
    headers: {
      'Content-Type': 'application/json'
    },
    data: JSON.stringify({
      "model": "llama3.2-vision",
      "stream": false,
      "messages": [
        {
          "role": "user",
          "content": prompt ? prompt : "Describe the image in great detail.",
          //"content": (
          //    "Extract all text from the image and return it as markdown.\n"
          //    "Do not describe the image or add extra text.\n"
          //    "Only return the text found in the image."
          //),
          "images": [base64_image]
        }
      ]
    })
  })
  //let buff = Buffer.from(result.data.images[0], 'base64');
  if(result.data && result.data.message)
    return result.data.message.content
  else
    return
}

module.exports = requestOllamaVision




### rpc services

start a bunch of llm rpc services?


In [ ]:

const {spawn, spawnSync} = require("child_process");
const path = require('path')

const ENVIRONMENTS = [
  void 0,
  {
    CHAT_PORT: 8181,
    DEFAULT_MODEL: 'Default',
  },
  {
    CHAT_PORT: 8282,
    DEFAULT_MODEL: 'Meta',
  },
  {
    CHAT_PORT: 8383,
    DEFAULT_MODEL: 'DeepSeek',
  },
  {
    CHAT_PORT: 8484,
    DEFAULT_MODEL: 'Qwen',
  },
  {
    CHAT_PORT: 8585,
    DEFAULT_MODEL: 'Code',
  },
  {
    CHAT_PORT: 8686,
    DEFAULT_MODEL: 'Mistral',
  },
]

function launchChats(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  for(let i = botIndex; i < ENVIRONMENTS.length; i++) {
    if(!ENVIRONMENTS[i]) {
      continue
    }

    spawn('node', ['--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'resume express chat service'], {
      env: Object.assign({}, process.env, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: [0, 1, 2],
      //detached: true,
      //shell: true,
      cwd: path.dirname(__dirname)
    })

    if(botIndex) {
      break
    }
  }
}

module.exports = launchChats



### stability connector

stable diffusion request?

imagine or generate an image based on the prompt. uses stable diffusion to generate an image.


In [ ]:
const fs = require('fs')
const path = require('path')
const {request} = require('gaxios')

const OUTPUT_PATH = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'stable-diffusion-webui/outputs')

async function doStableRequest(prompt) {
  let width = 1024
  if(prompt.includes('View360')) {
    width = 2048
  }
  let height = 1024
  let specificHeight = prompt.match(/--height=(\d+)/)
  let specificWidth = prompt.match(/--width=(\d+)/)
  if(specificHeight) {
    height = parseInt(specificHeight)
    prompt = prompt.replace(/--height=(\d+)/, '')
  }
  if(specificWidth) {
    width = parseInt(specificWidth)
    prompt = prompt.replace(/--width=(\d+)/, '')
  }
  try {
    let result = await request({
      url: 'http://127.0.0.1:7860/sdapi/v1/txt2img',
      method: 'POST',
      headers: {
        'Content-Type': 'application/json'
      },
      data: JSON.stringify({
        prompt: prompt,
        negative_prompt: 'bad hands, bad feet, bad faces, bad eyes, bad anatomy, extra limbs, missing limbs, tattoo, statue, picture frame, anime, cartoon, signature, abstract',
        save_images: true,
        "width": width,
        "height": height,
        "steps": 30,
        tiling: false,
      })
    })
    let seed = JSON.parse(result.data.info).seed
    let buff = Buffer.from(result.data.images[0], 'base64');
    let now = new Date()
    let folderName = now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0') + '-' + String(now.getDate()).padStart(2, '0')
    let stablePath = path.join(OUTPUT_PATH, 'txt2img-images', folderName)
    let imagePath
    if(fs.existsSync(stablePath)) {
      let images = fs.readdirSync(stablePath)
      for(let i = 0; i < images.length; i++) {
        if(images[i].match('-' + seed + '-')) {
          imagePath = path.join('txt2img-images', folderName, images[i])
          break
        }
      }
    }
    return {seed, image: buff, imagePath, prompt}
  } catch (e) {
    console.error(e)
  }
}

module.exports = {
  doStableRequest,
  OUTPUT_PATH
}




### remove background

remove an image background and return the image mask. useful for combining multiple images together.

remove background image?

mask image?


In [ ]:
const fs = require('fs')
const path = require('path')
const {request} = require('gaxios')
var crypto = require('crypto')
const sharp = require('sharp')

const OUTPUT_PATH = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'stable-diffusion-webui/outputs')

async function doBackgroundMask(image) {

  let base64_image
  if(typeof image == 'string') {
    if(image.startsWith('data:image/'))
      image = image.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(image.includes('://')) {
      let result = await request({
        url: image,
        method: 'GET',
      })
      base64_image = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(image)) {
      base64_image = Buffer.from(image, 'base64').toString('base64')
    } else {
      base64_image = fs.readFileSync(image).toString('base64')
    }  
  } else {
    base64_image = image.toString('base64')
  }

  try {
    let result = await request({
      url: 'http://127.0.0.1:7860/rembg',
      method: 'POST',
      headers: {
        'Content-Type': 'application/json'
      },
      data: JSON.stringify({
        input_image: base64_image,
        model: 'u2net',
        return_mask: true,
        //"alpha_matting": false,
        //"alpha_matting_foreground_threshold": 240,
        //"alpha_matting_background_threshold": 10,
        //"alpha_matting_erode_size": 10
      })
    })
    let buff = Buffer.from(result.data.image, 'base64')
    let seed = parseInt(crypto.randomBytes(8).toString('hex'), 16).toString()
    let now = new Date()
    let folderName = now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0') + '-' + String(now.getDate()).padStart(2, '0')
    let stablePath = path.join(OUTPUT_PATH, 'extras-images', folderName)
    if(!fs.existsSync(stablePath)) {
      fs.mkdirSync(stablePath)
    }
    let highestCount = 0
    let images = fs.readdirSync(stablePath)
    for(let i = 0; i < images.length; i++) {
      let index = parseInt(images[i].split(/[\.-]/gi)[0])
      if(!isNaN(index) && index > highestCount) {
        highestCount = index
      }
    }
    let imagePath = path.join('extras-images', folderName, String(highestCount).padStart(5, '0') + '-' + seed + '.png')
    fs.writeFileSync(path.join(OUTPUT_PATH, imagePath), buff)
    let imageObj = await sharp(buff)
    await imageObj.negate().toFile(path.join(OUTPUT_PATH, 'extras-images', folderName, String(highestCount).padStart(5, '0') + '-' + seed + '-negative.png'))
    return {seed, image: buff, imagePath}
  } catch (e) {
    console.error(e)
  }
}

module.exports = {
  doBackgroundMask,
  OUTPUT_PATH
}




### inpaint fill

inpaint an image by providing a subject and a mask. fill in the black part of the mask with a new generated image.

inpaint mask?



In [ ]:
const fs = require('fs')
const path = require('path')
const {request} = require('gaxios')

const OUTPUT_PATH = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'stable-diffusion-webui/outputs')

async function doInpaintMask(image, mask, prompt) {
  let width = 1024
  
  if(!prompt) {
    // TODO: image 2 image with ollama vision?
    return
  }

  if(prompt.includes('View360')) {
    width = 2048
  }

  let base64_image
  if(typeof image == 'string') {
    if(image.startsWith('data:image/'))
      image = image.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(image.includes('://')) {
      let result = await request({
        url: image,
        method: 'GET',
      })
      base64_image = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(image)) {
      base64_image = Buffer.from(image, 'base64').toString('base64')
    } else {
      base64_image = fs.readFileSync(image).toString('base64')
    }  
  } else {
    base64_image = image.toString('base64')
  }

  let base64_mask
  if(typeof mask == 'string') {
    if(mask.startsWith('data:image/'))
      mask = mask.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(mask.includes('://')) {
      let result = await request({
        url: mask,
        method: 'GET',
      })
      base64_mask = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(mask)) {
      base64_mask = Buffer.from(mask, 'base64').toString('base64')
    } else {
      base64_mask = fs.readFileSync(mask).toString('base64')
    }  
  } else {
    base64_mask = mask.toString('base64')
  }

  try {
    let result = await request({
      url: 'http://127.0.0.1:7860/sdapi/v1/img2img',
      method: 'POST',
      headers: {
        'Content-Type': 'application/json'
      },
      data: JSON.stringify({
        prompt: prompt,
        negative_prompt: 'bad hands, bad feet, bad faces, bad eyes, bad anatomy, extra limbs, missing limbs, tattoo, statue, picture frame, anime, cartoon, signature, abstract',
        save_images: true,
        "width": width,
        "height": 1024,
        "steps": 30,
        tiling: false,
        init_images: [base64_image],
        //denoising_strength: 0.35,
        mask: base64_mask,
        mask_blur: 4,
        inpainting_fill: 1,
        inpaint_full_res: false,
        inpaint_full_res_padding: 32,
        inpainting_mask_invert: 1,
      })
    })
    
    let seed = JSON.parse(result.data.info).seed
    let buff = Buffer.from(result.data.images[0], 'base64');
    let now = new Date()
    let folderName = now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0') + '-' + String(now.getDate()).padStart(2, '0')
    let stablePath = path.join(OUTPUT_PATH, 'img2img-images', folderName)
    let imagePath
    if(fs.existsSync(stablePath)) {
      let images = fs.readdirSync(stablePath)
      for(let i = 0; i < images.length; i++) {
        if(images[i].match('-' + seed + '-')) {
          imagePath = path.join('img2img-images', folderName, images[i])
          break
        }
      }
    }
    return {seed, image: buff, imagePath, prompt}

  } catch (e) {
    console.error(e)
  }
}

module.exports = {
  doInpaintMask,
  OUTPUT_PATH
}




### image2image

convert the style of an input image to another image, add details, change painting styles.

image 2 image?


In [ ]:
const fs = require('fs')
const path = require('path')
const {request} = require('gaxios')

const OUTPUT_PATH = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'stable-diffusion-webui/outputs')

async function doImage2Image(image, prompt) {
  let width = 1024
  if(prompt.includes('View360')) {
    width = 2048
  }

  let base64_image
  if(typeof image == 'string') {
    if(image.startsWith('data:image/'))
      image = image.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(image.includes('://')) {
      let result = await request({
        url: image,
        method: 'GET',
      })
      base64_image = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(image)) {
      base64_image = Buffer.from(image, 'base64').toString('base64')
    } else {
      base64_image = fs.readFileSync(image).toString('base64')
    }  
  } else {
    base64_image = image.toString('base64')
  }

  try {
    let result = await request({
      url: 'http://127.0.0.1:7860/sdapi/v1/img2img',
      method: 'POST',
      headers: {
        'Content-Type': 'application/json'
      },
      data: JSON.stringify({
        prompt: prompt,
        negative_prompt: 'bad hands, bad feet, bad faces, bad eyes, bad anatomy, extra limbs, missing limbs, tattoo, statue, picture frame, anime, cartoon, signature, abstract',
        save_images: true,
        "width": width,
        "height": 1024,
        "steps": 30,
        tiling: false,
        init_images: [base64_image],
      })
    })
    
    let seed = JSON.parse(result.data.info).seed
    let buff = Buffer.from(result.data.images[0], 'base64');
    let now = new Date()
    let folderName = now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0') + '-' + String(now.getDate()).padStart(2, '0')
    let stablePath = path.join(OUTPUT_PATH, 'img2img-images', folderName)
    let imagePath
    if(fs.existsSync(stablePath)) {
      let images = fs.readdirSync(stablePath)
      for(let i = 0; i < images.length; i++) {
        if(images[i].match('-' + seed + '-')) {
          imagePath = path.join('img2img-images', folderName, images[i])
          break
        }
      }
    }
    return {seed, image: buff, imagePath, prompt}

  } catch (e) {
    console.error(e)
  }
}

module.exports = {
  doImage2Image,
  OUTPUT_PATH
}




### google whisk

combine multiple images and vision prompts into a single output image, select a subject, a scene, and an art style to whisk together.

whisk images?

TODO: center the subject using the center of white and crop to be square so it doesn't distort


In [ ]:
const fs = require('fs')
const { request } = require('gaxios')
const requestOllamaVision = importer.import('request ollama vision')
const selectModel = importer.import('select llm')
const {doStableRequest} = importer.import('stable diffusion request')
const {doImage2Image} = importer.import('image 2 image')
const {doBackgroundMask} = importer.import('mask image')
const {doInpaintMask} = importer.import('inpaint mask')
// TODO: use the above functions in combination to whisk together a set of images

async function whiskImages(subject, scene, style, short) {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Default')

  let subjectString
  let subjectShort
  let base64_subject
  if(typeof subject == 'string') {
    if(subject.startsWith('data:image/')) {
      subject = subject.replace(/^data:image\/.*?;base64,/gi, '')
      base64_subject = Buffer.from(subject, 'base64').toString('base64')
    } else if(subject.includes('://')) {
      let result = await request({
        url: subject,
        method: 'GET',
      })
      base64_subject = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(subject)) {
      subjectString = subject
    } else {
      base64_subject = fs.readFileSync(subject).toString('base64')
    }  
  } else if(subject) {
    base64_subject = subject.toString('base64')
  }


  let sceneString
  let sceneShort
  let base64_scene
  if(typeof scene == 'string') {
    if(scene.startsWith('data:image/')) {
      scene = scene.replace(/^data:image\/.*?;base64,/gi, '')
      base64_scene = Buffer.from(scene, 'base64').toString('base64')
    } else if(scene.includes('://')) {
      let result = await request({
        url: scene,
        method: 'GET',
      })
      base64_scene = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(scene)) {
      sceneString = scene
    } else {
      base64_scene = fs.readFileSync(scene).toString('base64')
    }  
  } else if (scene) {
    base64_scene = scene.toString('base64')
  }


  let styleString
  let styleShort
  let base64_style
  if(typeof style == 'string') {
    if(style.startsWith('data:image/')) {
      style = style.replace(/^data:image\/.*?;base64,/gi, '')
      base64_style = Buffer.from(style, 'base64').toString('base64')
    } else if(style.includes('://')) {
      let result = await request({
        url: style,
        method: 'GET',
      })
      base64_style = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(style)) {
      styleString = style
    } else {
      base64_style = fs.readFileSync(style).toString('base64')
    }  
  } else if(style) {
    base64_style = style.toString('base64')
  }

  // TODO: if passing in an image, ask ollama vision for a description, 
  //   if passing in a description use it to generate the next image
  if(!subjectString && base64_subject) {
    subjectString = await requestOllamaVision('data:image/png;base64,' + base64_subject, 'Describe the foreground subject of the image in one short sentence.')
  }
  if(short && subjectString) {
    subjectShort = await promptModel('Summarize this sentence into four or five words:\n' + subjectString + '\nOnly return the summary, no title or explanation.')
  }

  if(!sceneString && base64_scene) {
    sceneString = await requestOllamaVision('data:image/png;base64,' + base64_scene, 'Describe the scenery in the image in one short sentence.')
  }
  if(short && sceneString) {
    sceneShort = await promptModel('Summarize this sentence into four or five words:\n' + sceneString + '\nOnly return the summary, no title or explanation.')
  }

  if(!styleString && base64_style) {
    styleString = await requestOllamaVision('data:image/png;base64,' + base64_style, 'Describe the art style of image in one short sentence.')
  }
  if(short && styleString) {
    styleShort = await promptModel('Summarize this sentence into four or five words:\n' + styleString + '\nOnly return the summary, no title or explanation.')
  }

  // TODO: if no scene, only subject and style, then just call image 2 image
  if(!base64_scene && !base64_style && !base64_subject) {
    // no images passed in, send directly to image generator
    if(short)
      return await doStableRequest(subjectShort + '\n' + sceneShort + '\n' + styleShort)
    else
      return await doStableRequest(subjectString + '\n' + sceneString + '\n' + styleString)
  } else if (base64_subject && !sceneString) {
    // subject and style process only, pass directly to image 2 image
    if(short)
      return await doImage2Image('data:image/png;base64,' + base64_subject, subjectShort + (styleShort ? ('\n' + styleShort) : ''))
    else
      return await doImage2Image('data:image/png;base64,' + base64_subject, subjectString + (styleString ? ('\n' + styleString) : ''))
  } else if (base64_scene && !subjectString) {
    // scene and style only, pass to image 2 image
    if(short)
      return await doImage2Image('data:image/png;base64,' + base64_scene, sceneShort + (styleShort ? ('\n' + styleShort) : ''))
    else
      return await doImage2Image('data:image/png;base64,' + base64_scene, sceneString + (styleString ? ('\n' + styleString) : ''))
  } else if (base64_subject && sceneString) {
    // TODO: extract mask on subject
    let maskObject = await doBackgroundMask('data:image/png;base64,' + base64_subject)
    let base64_mask = maskObject.image.toString('base64')

    // TODO: combine subject with new scene
    let inpaintObject
    if(short)
      inpaintObject = await doInpaintMask(
        'data:image/png;base64,' + base64_subject, 
        'data:image/png;base64,' + base64_mask, 
        sceneShort)
    else
      inpaintObject = await doInpaintMask(
        'data:image/png;base64,' + base64_subject, 
        'data:image/png;base64,' + base64_mask, 
        sceneString)
    
    // Drop out early if there is no style specified, just do the proper inpainting
    if(!styleString) {
      return inpaintObject
    }

    let base64_inpaint = inpaintObject.image.toString('base64')
    

    // TODO: generate final image in new style
    if(short)
      return await doImage2Image('data:image/png;base64,' + base64_inpaint, styleShort + '\n' + subjectShort + '\n' + sceneShort)
    else
      return await doImage2Image('data:image/png;base64,' + base64_inpaint, styleString + '\n' + subjectString + '\n' + sceneString)
  } else {
    console.error('Missing components: ')
    return {}
  }
}

module.exports = whiskImages

